In [1]:
import pandas as pd
import cv2 as cv
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import sklearn.model_selection as model_selection
import keras.utils.np_utils as np_utils
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# declare path to needed files
FOLDER_PATH='dataset_mini'

# declare path to images folder
IMAGES_PATH=f'{FOLDER_PATH}/images'

In [3]:
# read the CSV files containing the images and their correspondent class/label
multiclass_df = pd.read_csv(f"{FOLDER_PATH}/multiclass.csv")
labels_df = pd.read_csv(f"{FOLDER_PATH}/labels.csv")

print('Number of total samples: ', multiclass_df.shape[0])
print('Number of labels: ', labels_df.shape[0])

multiclass_df["id"] = multiclass_df["id"].apply(lambda x: f"{x}.png")
multiclass_df["attribute_ids"] = multiclass_df["attribute_ids"].astype(str)

display(multiclass_df.head())
display(labels_df.head())

Number of total samples:  22087
Number of labels:  50


,id,attribute_ids
0,1000fe2e667721fe.png,51
1,10041eb49b297c08.png,51
2,100501c227f8beea.png,13
3,1008abd71f3ed5bc.png,70
4,100a0dcde728cb36.png,51


,attribute_id,attribute_name
0,0,culture::abruzzi
1,1,culture::achaemenid
2,2,culture::aegean
3,9,culture::akkadian
4,10,culture::alexandria-hadra


In [4]:
# define number of words for vocabulary
NUMBER_OF_WORDS = 100

# create paths for all images
input_images = [f'{IMAGES_PATH}/{x}' for x in multiclass_df["id"]]

# function to open an image using its path
def openImage(filename):
    image = cv.imread(filename)
    try:
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    except:
        print(" --(!) Error reading image ", filename)
        return None
    return image

In [5]:
# compute all descriptors and keypoints for all images
detector = cv.KAZE_create()
allDescriptors = []
allKeypoints = []
for name in tqdm(input_images):
    img = openImage(name)
    if img is None:
        continue
    keypoints, descriptors = detector.detectAndCompute(img, None)
    if descriptors is not None:
        allDescriptors.extend(descriptors)
    allKeypoints.append(keypoints)

100%|██████████| 22087/22087 [26:29<00:00, 13.89it/s]


In [6]:
# instantiate Bag Of Words trainer
bowTrainer = cv.BOWKMeansTrainer(NUMBER_OF_WORDS)

# create visual vocabulary, clustering image descriptors using K-Means
vocab = bowTrainer.cluster(np.array(allDescriptors))

In [7]:
# create Bag Of Words extractor class
matcher = cv.FlannBasedMatcher()
bowExtractor = cv.BOWImgDescriptorExtractor(detector, matcher)
bowExtractor.setVocabulary(vocab)

print(vocab.shape)

(100, 64)


In [8]:
# extract all Bag Of Words descriptors for each image
allBowDescriptors = []
for name, keypoint in tqdm(zip(input_images, allKeypoints)):
    img = openImage(name)
    desc = bowExtractor.compute(img, keypoint)
    
    desc = np.zeros(NUMBER_OF_WORDS) if desc is None else desc.squeeze()
    allBowDescriptors.append(desc)

22087it [18:47, 19.60it/s]


In [9]:
# split the images that are going to be used for training and for testing
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    allBowDescriptors,
    multiclass_df["attribute_ids"].tolist(),
    test_size = 0.2
)

In [10]:
print(np.array(allBowDescriptors).shape)

(22087, 100)


In [11]:
#print(allBowDescriptors)
#outfile = open('allBowDescriptors', 'wb')
#pickle.dump(allBowDescriptors, outfile)
#outfile.close()

### SVM Classifier

In [12]:
# import classes from SKLearn
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import multilabel_confusion_matrix

In [13]:
# create SVM classifier instance, and check the hyper parameters
svm = SVC()
svm.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [14]:
# do hyper parameter tuning with the SVM, using cross validation
kernel = ['linear', 'poly', 'rbf']
c = [0.01, 0.1, 1]
gamma = [0.01, 0.1, 1]
epsilon = [0.01, 0.1, 1]
param_grid = {'C': c, 'gamma' : gamma, 'kernel': kernel}

search = GridSearchCV(svm, param_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, verbose=1)

In [15]:
def countNone(array):
    return sum(1 for i in array if i is None)

print(f'{countNone(X_train)} null elements in X_train')
print(f'{countNone(y_train)} null elements in y_train\n')
       
print(f'{countNone(X_test)} null elements in X_test')
print(f'{countNone(y_test)} null elements in y_test\n')

0 null elements in X_train
0 null elements in y_train

0 null elements in X_test
0 null elements in y_test



In [16]:
import time

# train the model with the training data
start = time.time()
search.fit(X_train, y_train)
end = time.time()
elapsed_time = end - start
print('Elapsed time: {}mins'.format(elapsed_time / 60))
print('Best score: {}'.format(search.best_score_))
print('Best parameters: {}'.format(search.best_params_))
svm_estimator = search.best_estimator_

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Elapsed time: 22.983833463986716mins
Best score: -563.7028821732519
Best parameters: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}


In [17]:
# test the classifier with the testing data
y_pred = svm_estimator.predict(X_test)

In [18]:
# create a multilabel confusion matrix
multilabel_confusion_matrix(y_test, y_pred, labels=list(multiclass_df["attribute_ids"]))

array([[[2119,  832],
        [ 701,  766]],

       [[2119,  832],
        [ 701,  766]],

       [[1240, 1334],
        [ 358, 1486]],

       ...,

       [[4286,    0],
        [ 132,    0]],

       [[1240, 1334],
        [ 358, 1486]],

       [[1240, 1334],
        [ 358, 1486]]], dtype=int64)

In [19]:
print(classification_report(y_test, y_pred, zero_division=0, labels=np.unique(multiclass_df["attribute_ids"])))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00        16
          10       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         3
          13       0.53      0.81      0.64      1844
          14       0.00      0.00      0.00        84
          15       0.00      0.00      0.00         5
          16       0.00      0.00      0.00         4
          17       0.00      0.00      0.00         5
          18       0.00      0.00      0.00        28
           2       0.00      0.00      0.00         1
          22       0.00      0.00      0.00         4
          23       0.00      0.00      0.00        52
          24       0.00      0.00      0.00         8
          25       0.00      0.00      0.00       132
          26       0.00      0.00      0.00        29
          27       0.00      0.00      0.00         4
          28       0.00    